In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

### For latter flights prediction model, we need one additional feature -- status of the former flight

Steps to add column -- `FORMER_FLIGHT_STATUS`
- Things we consider:
    - For any given flight -- `FORMER_FLIGHT_STATUS` = Status of the preceding flight **just before the given flight** on **same day** and **same origin - destination**.
- We first sort the data, according scheduled arrival time. We reset index after that.
- Then for every given row, we figure its `FORMER_FLIGHT_STATUS` based on above consideration.


In [2]:

flight_data = pd.read_csv('./dataset/merged_data/flight_weather_hourly_jan2022_dec2023_for_training.csv')
flight_data['SCH_ARR_TIME'] = pd.to_datetime(flight_data['SCH_ARR_TIME'])
flight_data['SCH_DEP_TIME'] = pd.to_datetime(flight_data['SCH_DEP_TIME'])
flight_data = flight_data.sort_values(by='SCH_ARR_TIME').reset_index(drop=True)
flight_data.head(10)

,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARR_TIME,ARR_DELAY,DIVERTED,SCH_DEP_TIME,SCH_ARR_TIME,ORGIN_WTH_temp,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,DEST_WTH_temp,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,DEST_WTH_conditions
0,6,WN,WN,MCO,"1,254.00",-26.00,0,2022-01-01 10:30:00,2022-01-01 13:20:00,74.00,0.00,0,NaN,0.00,6.90,200.00,2.90,9.90,NaN,Clear,48.00,0.00,0,NaN,0.00,3.60,8.00,100.00,9.80,NaN,Overcast
1,6,UA,OO,ORD,"1,307.00",-25.00,0,2022-01-01 10:40:00,2022-01-01 13:32:00,36.10,0.00,0,rain,0.02,21.00,20.00,100.00,9.90,NaN,Overcast,47.90,0.00,0,NaN,0.00,0.40,358.00,100.00,9.70,NaN,Overcast
2,6,B6,B6,MCO,"1,618.00",22.00,0,2022-01-01 13:13:00,2022-01-01 15:56:00,83.00,0.00,0,NaN,0.00,9.90,199.00,4.30,9.90,NaN,Clear,47.70,0.00,0,NaN,0.00,7.90,311.00,100.00,7.80,NaN,Overcast
3,6,B6,B6,JFK,"2,335.00",36.00,0,2022-01-01 21:45:00,2022-01-01 22:59:00,52.80,0.15,100,rain,0.00,8.10,40.00,100.00,2.20,NaN,Rain,37.90,0.02,100,rain,0.00,6.10,303.00,100.00,6.80,NaN,Rain
4,7,B6,B6,JFK,938.00,-12.00,0,2022-01-02 08:29:00,2022-01-02 09:50:00,52.10,0.00,0,NaN,0.00,0.00,0.00,100.00,5.90,NaN,Overcast,25.00,0.00,0,snow,0.01,13.80,303.00,100.00,1.20,NaN,Overcast
5,7,AA,MQ,ORD,"1,347.00",31.00,0,2022-01-02 10:25:00,2022-01-02 13:16:00,22.30,0.00,0,NaN,0.00,11.30,340.00,100.00,5.90,NaN,Overcast,23.00,0.01,100,snow,0.01,10.20,283.00,100.00,4.90,NaN,Snow
6,7,UA,OO,ORD,"1,420.00",48.00,0,2022-01-02 10:40:00,2022-01-02 13:32:00,23.50,0.00,0,NaN,0.00,11.30,330.00,90.30,6.70,NaN,Overcast,23.00,0.00,0,snow,0.01,11.20,301.00,100.00,8.50,NaN,Overcast
7,7,DL,9E,JFK,"1,712.00",180.00,0,2022-01-02 12:55:00,2022-01-02 14:12:00,57.20,0.00,0,NaN,0.00,10.10,243.00,90.70,9.40,NaN,Overcast,23.00,0.00,0,snow,0.01,11.20,301.00,100.00,8.50,NaN,Overcast
8,7,B6,B6,MCO,"1,700.00",64.00,0,2022-01-02 13:13:00,2022-01-02 15:56:00,82.10,0.00,0,NaN,0.00,9.60,217.00,48.60,9.90,NaN,Partially cloudy,23.00,0.00,0,snow,0.01,10.10,273.00,100.00,3.10,NaN,Overcast
9,7,AA,MQ,ORD,"2,047.00",35.00,0,2022-01-02 17:25:00,2022-01-02 20:12:00,25.70,0.00,0,NaN,0.00,6.40,329.00,24.20,9.90,NaN,Partially cloudy,24.00,0.00,0,snow,0.01,6.80,338.00,99.90,9.90,NaN,Overcast


In [3]:
flight_data.shape
row = flight_data.iloc[4]
previous_flight_rows = flight_data[(flight_data.index < row.name)\
                                & (flight_data['SCH_ARR_TIME'].dt.date == row['SCH_ARR_TIME'].date())\
                                & (flight_data['ORIGIN'] == row['ORIGIN'])]
previous_flight_rows.shape[0]
row.name
flight_data.index

(6773, 31)

KeyError: 'DEST'

In [ ]:
def get_former_flight_status(row: pd.Series):
    previous_flight_rows = flight_data[(flight_data.index < row.name)\
                                & (flight_data['SCH_ARR_TIME'].dt.date == row['SCH_ARR_TIME'].date())\
                                & (flight_data['ORIGIN'] == row['ORIGIN'])\
                                & (flight_data['DEST'] == row['DEST'])]
    if previous_flight_rows.shape[0] <= 0:
        return np.nan
    else:
        arr_delay = previous_flight_rows.iloc[-1]['ARR_DELAY']
        if arr_delay < -5:
            return 'early'
        elif arr_delay > 5:
            return 'late'
        else:
            return 'on-time'

In [ ]:
flight_data['FORMER_FLIGHT_STATUS'] = flight_data.apply(get_former_flight_status, axis=1)

,ORIGIN,DEST,SCH_ARR_TIME,FORMER_FLIGHT_STATUS,ARR_DELAY
0,MCO,SYR,2022-01-01 13:20:00,NaN,-26.00
1,ORD,SYR,2022-01-01 13:32:00,NaN,-25.00
2,MCO,SYR,2022-01-01 15:56:00,early,22.00
3,ORD,SYR,2022-01-01 17:36:00,early,NaN
4,ORD,SYR,2022-01-01 18:15:00,on-time,NaN
5,ORD,SYR,2022-01-01 20:14:00,on-time,NaN
6,ORD,SYR,2022-01-01 20:52:00,on-time,NaN
7,JFK,SYR,2022-01-01 22:59:00,NaN,36.00
8,JFK,SYR,2022-01-02 09:50:00,NaN,-12.00
9,ORD,SYR,2022-01-02 13:16:00,NaN,31.00


In [ ]:
flight_data[['ORIGIN', 'DEST', 'SCH_ARR_TIME','FORMER_FLIGHT_STATUS', 'ARR_DELAY']].head(10)

,ORIGIN,DEST,SCH_ARR_TIME,FORMER_FLIGHT_STATUS,ARR_DELAY
0,MCO,SYR,2022-01-01 13:20:00,NaN,-26.00
1,ORD,SYR,2022-01-01 13:32:00,NaN,-25.00
2,MCO,SYR,2022-01-01 15:56:00,early,22.00
3,ORD,SYR,2022-01-01 17:36:00,early,NaN
4,ORD,SYR,2022-01-01 18:15:00,on-time,NaN
5,ORD,SYR,2022-01-01 20:14:00,on-time,NaN
6,ORD,SYR,2022-01-01 20:52:00,on-time,NaN
7,JFK,SYR,2022-01-01 22:59:00,NaN,36.00
8,JFK,SYR,2022-01-02 09:50:00,NaN,-12.00
9,ORD,SYR,2022-01-02 13:16:00,NaN,31.00


In [ ]:
flight_data.to_csv('./dataset/merged_data/latter_flight_data.csv', index=False)